In [1]:
import torch
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from tqdm import tqdm
from torch.nn import functional as F
import matplotlib.pyplot as plt

from torch.optim.lr_scheduler import LambdaLR

In [2]:
import torch
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from tqdm import tqdm
from torch.nn import functional as F
import matplotlib.pyplot as plt

from torch.optim.lr_scheduler import LambdaLR

### Load Data

In [3]:
df = pd.read_csv("./Students_Grading_Dataset.csv")

## Manually select Cols (attribute)

In [4]:
unimportant_attribute = ['Student_ID', 'First_Name', 'Last_Name', 'Email', 'Participation_Score']

filtered_df = df.drop(unimportant_attribute, axis=1)
filtered_df

,Gender,Age,Department,Attendance (%),Midterm_Score,Final_Score,Assignments_Avg,Quizzes_Avg,Projects_Score,Total_Score,Grade,Study_Hours_per_Week,Extracurricular_Activities,Internet_Access_at_Home,Parent_Education_Level,Family_Income_Level,Stress_Level (1-10),Sleep_Hours_per_Night
0,Female,22,Engineering,52.29,55.03,57.82,84.22,74.06,85.90,56.09,F,6.2,No,Yes,High School,Medium,5,4.7
1,Male,18,Engineering,97.27,97.23,45.80,NaN,94.24,55.65,50.64,A,19.0,No,Yes,NaN,Medium,4,9.0
2,Male,24,Business,57.19,67.05,93.68,67.70,85.70,73.79,70.30,D,20.7,No,Yes,Master's,Low,6,6.2
3,Female,24,Mathematics,95.15,47.79,80.63,66.06,93.51,92.12,61.63,A,24.8,Yes,Yes,High School,High,3,6.7
4,Female,23,CS,54.18,46.59,78.89,96.85,83.70,68.42,66.13,F,15.4,Yes,Yes,High School,High,2,7.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,Male,19,Business,NaN,82.15,60.33,80.09,99.32,58.42,85.21,D,25.5,No,Yes,High School,Low,10,8.3
4996,Male,19,Business,65.11,86.31,49.80,NaN,88.08,60.87,95.96,C,5.0,No,Yes,NaN,Medium,4,4.0
4997,Female,24,CS,87.54,63.55,64.21,94.28,50.19,82.65,54.25,A,24.8,Yes,No,High School,Medium,4,6.3
4998,Male,23,CS,92.56,79.79,94.28,81.20,61.18,94.29,55.84,A,16.1,Yes,Yes,Bachelor's,Low,1,8.4


In [5]:
category_vars = ['Gender', 'Department', 'Grade', 'Extracurricular_Activities', 'Internet_Access_at_Home', 'Parent_Education_Level', 'Family_Income_Level']
# numerical_score_vars = ['Attendance (%)', 'Midterm_Score', 'Final_Score', 'Assignments_Avg', 'Quizzes_Avg', 'Participation_Score', 'Projects_Score', 'Total_Score', 'Stress_Level (1-10)']
numerical_score_vars = ['Attendance (%)', 'Midterm_Score', 'Final_Score', 'Assignments_Avg', 'Quizzes_Avg', 'Projects_Score', 'Total_Score']
numerical_scalar_vars = list(set(filtered_df.columns) - set(category_vars) - set(numerical_score_vars))
numerical_scalar_vars

['Sleep_Hours_per_Night', 'Stress_Level (1-10)', 'Study_Hours_per_Week', 'Age']

## Separate rows with-Nan and without-Nan

In [6]:
nan_rows = filtered_df.isna().any(axis=1)

# Nan rows
df_nan = filtered_df[nan_rows]
print(f"row with Nan: {df_nan.shape}")

# Complete rows
df_complete = filtered_df[~nan_rows]
print(f"row without Nan: {df_complete.shape}")

row with Nan: (2419, 18)
row without Nan: (2581, 18)


In [7]:
df_train, df_valid, _, _ = train_test_split(df_complete, df_complete, test_size=0.3, random_state=0)

df_train_id = [i for i in range(len(df_train))]
df_valid_id = [i for i in range(len(df_valid))]

print(f"df_train: {df_train.shape}")
print(f"df_valid: {df_valid.shape}")

# print(f"df_train_id: {df_train_id}")
# print(f"df_valid_id: {df_valid_id}")


df_train: (1806, 18)
df_valid: (775, 18)


## Preprocessing: 
1. category to numerical
2. max-min norm

In [8]:
def category_to_numerical(data):
    le = LabelEncoder()
    le.fit(data)
    num_data = le.transform(data)
    
    return num_data, le

def max_min_norm_score(data, train_params = None, process_type = 'train'):
    
    if process_type == 'train':
        data_max = 100
        data_min = 0
    else:
        data_max = 100
        data_min = 0
        
    norm_data = (data - data_min) / (data_max - data_min)    
    
    if process_type == 'train':
        return norm_data, data_max, data_min
    else:
        return norm_data
    
def max_min_norm_scalar(data, train_params = None, process_type = 'train'):
    
    if process_type == 'train':
        data_max = 10
        data_min = 0
    else:
        data_max = 10
        data_min = 0
        
    norm_data = (data - data_min) / (data_max - data_min)    
    
    if process_type == 'train':
        return norm_data, data_max, data_min
    else:
        return norm_data

    
def preprocessing(df, train_params = None, process_type = 'train'):
    
    new_df = pd.DataFrame()
    
    if process_type == 'train':
        train_params = {}
        category_var_len = {}

    # Category 
    for cat_name in category_vars:
        cat_var = df[cat_name]
        if process_type == 'train':
            cat_var, le = category_to_numerical(cat_var)
            train_params[f'{cat_name}_le'] = le
            category_var_len[f'{cat_name}'] = len(np.unique(cat_var))
        else:
            cat_var = train_params[f'{cat_name}_le'].transform(cat_var)
        new_df[f'{cat_name}'] = cat_var
    
    # Numerical score
    for num_name in numerical_score_vars:
        num_var = df[num_name]
        if process_type == 'train':
            num_var, data_max, data_min = max_min_norm_score(num_var, process_type = 'train')
            train_params[num_name] = [data_max, data_min]
        else:
            num_var = max_min_norm_score(num_var, train_params, process_type = 'valid')
        new_df[num_name] = num_var.values
    
    # Numerical scalar
    for num_name in numerical_scalar_vars:
        num_var = df[num_name]
        num_var = np.log(num_var)
        if process_type == 'train':
            num_var, data_max, data_min = max_min_norm_scalar(num_var, process_type = 'train')
            train_params[num_name] = [data_max, data_min]
        else:
            num_var = max_min_norm_scalar(num_var, train_params, process_type = 'valid')
        new_df[num_name] = num_var.values
        
        
    if process_type == 'train':
        return new_df, train_params, category_var_len
    else:
        return new_df


In [9]:
processed_df_train, train_params, category_var_len = preprocessing(df_train, process_type = 'train')
# train_params
print(f"category_var_len: {category_var_len}")
print(f"processed_df_train: {processed_df_train.shape}")

cols_name = processed_df_train.columns
processed_df_train.head()

category_var_len: {'Gender': 2, 'Department': 4, 'Grade': 5, 'Extracurricular_Activities': 2, 'Internet_Access_at_Home': 2, 'Parent_Education_Level': 4, 'Family_Income_Level': 3}
processed_df_train: (1806, 18)


,Gender,Department,Grade,Extracurricular_Activities,Internet_Access_at_Home,Parent_Education_Level,Family_Income_Level,Attendance (%),Midterm_Score,Final_Score,Assignments_Avg,Quizzes_Avg,Projects_Score,Total_Score,Sleep_Hours_per_Night,Stress_Level (1-10),Study_Hours_per_Week,Age
0,1,1,1,1,1,0,1,0.8989,0.4255,0.5045,0.5528,0.7634,0.5783,0.7426,0.212823,0.138629,0.332504,0.317805
1,1,3,2,0,1,3,2,0.5562,0.8190,0.4543,0.5380,0.9215,0.6371,0.8995,0.193152,0.069315,0.177495,0.313549
2,1,1,4,0,1,1,1,0.6309,0.6057,0.5680,0.5342,0.8288,0.5671,0.5943,0.158924,0.194591,0.293916,0.294444
3,1,2,2,1,1,1,2,0.6077,0.9869,0.4550,0.5771,0.5339,0.5516,0.8337,0.210413,0.230259,0.282138,0.294444
4,0,2,1,0,1,1,2,0.9590,0.8459,0.6858,0.5204,0.6389,0.6554,0.8854,0.218605,0.179176,0.184055,0.304452


In [10]:
processed_df_valid = preprocessing(df_valid, train_params, process_type = 'valid')
print(f"processed_df_valid: {processed_df_valid.shape}")
processed_df_valid.head()

processed_df_valid: (775, 18)


,Gender,Department,Grade,Extracurricular_Activities,Internet_Access_at_Home,Parent_Education_Level,Family_Income_Level,Attendance (%),Midterm_Score,Final_Score,Assignments_Avg,Quizzes_Avg,Projects_Score,Total_Score,Sleep_Hours_per_Night,Stress_Level (1-10),Study_Hours_per_Week,Age
0,1,3,2,0,1,2,1,0.5154,0.6404,0.5002,0.8728,0.9699,0.8258,0.8888,0.214007,0.207944,0.283908,0.294444
1,1,1,1,1,1,2,1,0.9970,0.9376,0.6142,0.5457,0.7576,0.5397,0.6468,0.141099,0.207944,0.194591,0.289037
2,1,3,2,0,1,3,0,0.6151,0.7315,0.4190,0.5537,0.5564,0.7862,0.5429,0.193152,0.219722,0.294969,0.317805
3,1,1,3,1,1,2,1,0.5228,0.8212,0.9962,0.5518,0.5023,0.8334,0.6564,0.185630,0.194591,0.319458,0.304452
4,1,2,0,0,1,0,2,0.9474,0.5340,0.6216,0.8329,0.7903,0.6250,0.6557,0.200148,0.109861,0.236085,0.313549


In [11]:
BATCH_SIZE = 256
MASK_RATIO = 30

In [12]:
def masking_helper(data, MASK_RATIO, seed = 42):
    np.random.seed(seed)

    rows, cols = data.shape
    
    # define mask/unmask ratio
    unmask_ratio = ((100 - MASK_RATIO) * cols) // 100
    mask_ratio = cols - unmask_ratio

    # create random index
    # shuff_idx = np.array([np.random.permutation(cols) for _ in range(rows)])
    shuff_idx = np.random.permutation(cols).reshape(1, cols)
    shuff_idx = np.repeat(shuff_idx, rows, axis = 0)

    # define mask/unmask idx
    mask_idx = shuff_idx[:, :mask_ratio]
    unmask_idx = shuff_idx[:, mask_ratio:]
    
    mask_idx.sort(axis=1)
    unmask_idx.sort(axis=1)
    
    # create new_data (contain unmask cols, but remove mask cols)
    new_data = np.zeros((rows, unmask_ratio))
    for i in range(rows):
        new_data[i] = data[i][unmask_idx[i]]
        
    return new_data, unmask_idx, mask_idx, unmask_ratio

def masking_fn(data, seed):
    sample_size = len(data)

    new_data, unmask_idx, mask_idx, unmask_ratio = masking_helper(data, MASK_RATIO, seed)
        
    X = [[],[],[],[]]
    Y = []

    for i in range(sample_size):
        X[0].append(new_data[i]) # unmask data
        X[1].append(unmask_idx[i]) # unmask id
        X[2].append(mask_idx[i]) # mask id
        X[3].append(np.ones(unmask_ratio)) # len = unmask, serve as random noisee in VAE
        Y.append(data[i][list(unmask_idx[i])+list(mask_idx[i])]) # label (unmask + mask)

    X[0] = torch.tensor(np.array(X[0])) # unmask data
    X[1] = torch.tensor(np.array(X[1])) # unmask id
    X[2] = torch.tensor(np.array(X[2])) # mask id
    X[3] = torch.tensor(np.array(X[3])) # latent space
    
    Y = torch.tensor(np.array(Y))
    

    return X, Y

In [13]:
def collate_fn_train(data_wtih_type):

    # data
    data = [dat[0] for dat in data_wtih_type]
    data = np.array(data)
    
    # data type: training/valid
    data_type = data_wtih_type[0][1]

    if data_type == 'training':
        seed = np.random.randint(3, size=1)
    else:
        seed = 42
        
    data, label = masking_fn(data, seed)

    return data, label

class TableDataset(Dataset):
    def __init__(self, data, data_type = 'training'):
        self.data = data
        self.data_type = data_type
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, id):
        dat = self.data.iloc[id, :]
        dat = np.array(dat)
        return dat, self.data_type
        

        

In [14]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

augmented_processed_df_train = pd.concat([processed_df_train, processed_df_train], axis = 0)
train_dataset = TableDataset(augmented_processed_df_train)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers = 4, shuffle=False, collate_fn = collate_fn_train)
# train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn = collate_fn_train)


valid_dataset = TableDataset(processed_df_valid, data_type = 'valid')
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, num_workers = 4, shuffle=False, collate_fn = collate_fn_train)

In [15]:
class Head(nn.Module):
    def __init__(self, num_head, encoder_emb_dim, dropout):
        super().__init__()
        self.num_head = num_head
        self.encoder_emb_dim = encoder_emb_dim

        # up_emb_size * 3 for qkv
        self.qkv_fn = nn.Linear(self.encoder_emb_dim, self.encoder_emb_dim * 3, bias=False)
        self.proj_qkv = nn.Linear(self.encoder_emb_dim, self.encoder_emb_dim, bias = False)
                
        # other
        self.att_dropout = nn.Dropout(dropout)
        
    def forward(self, x):

        batch_size, num_vars, _ = x.shape

        # qkv: up_emb_size * 3
        x = self.qkv_fn(x) 
        q, k, v = x.split(self.encoder_emb_dim, dim = 2) 
        
        # split head: each shape = [batch_size, num_head, num_vars(seq_len), head_size = 8]
        q = q.view(batch_size, num_vars, self.num_head, self.encoder_emb_dim // self.num_head).transpose(1, 2)
        k = k.view(batch_size, num_vars, self.num_head, self.encoder_emb_dim // self.num_head).transpose(1, 2)
        v = v.view(batch_size, num_vars, self.num_head, self.encoder_emb_dim // self.num_head).transpose(1, 2)
        
        # attention matrix calculation: [batch_size, num_head, num_vars(seq_len), num_vars]
        att = (q @ k.transpose(-2,-1)) * (1 / torch.sqrt(torch.ones([1]).to(device) * k.size(-1)))
        att = F.softmax(att, dim = -1)
        att = self.att_dropout(att)
        
        # att matrix * V: [batch_size, num_head, num_vars(seq_len), head_size]
        out = att @ v
        out = out.transpose(1,2).contiguous().view(batch_size, num_vars, self.encoder_emb_dim)
        out = self.proj_qkv(out)
        
        return out

class MLP(nn.Module):

    def __init__(self, transformer_emb_size, dropout):
        super().__init__()
        self.up    = nn.Linear(transformer_emb_size, transformer_emb_size * 3)
        self.relu  = nn.ReLU()
        self.down  = nn.Linear(transformer_emb_size * 3, transformer_emb_size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        x = self.up(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.down(x)
        
        return x
    
class Block(nn.Module):
    def __init__(self, encoder_emb_dim, num_head, dropout):
        super(Block, self).__init__()
        self.ln_head = nn.LayerNorm(encoder_emb_dim)
        self.head = Head(num_head, encoder_emb_dim, dropout)
        self.dropout_head = nn.Dropout(dropout)

        self.ln_mlp = nn.LayerNorm(encoder_emb_dim)
        self.mlp = MLP(encoder_emb_dim, dropout)
        self.dropout_mlp = nn.Dropout(dropout)

    def forward(self, x):
        
        # att 
        ori_x = x 
        x = self.head(x)
        x = self.dropout_head(x)
        x = self.ln_head(ori_x + x)
        
        # linear 
        ori_x = x 
        x = self.mlp(x)
        x = self.dropout_mlp(x)
        x = self.ln_mlp(ori_x + x)

        return x

class Transformer(nn.Module):
    def __init__(self, encoder_emb_dim, layers, num_head, dropout):
        super().__init__()
        self.blocks = nn.ModuleList(Block(encoder_emb_dim, num_head, dropout) for _ in range(layers))
        
    def forward(self, x):

        for block in self.blocks:
            x = block(x)
            
        return x

In [16]:
category_vars = ['Gender', 'Department', 'Grade', 'Extracurricular_Activities', 'Internet_Access_at_Home', 'Parent_Education_Level', 'Family_Income_Level']
numerical_score_vars = ['Attendance (%)', 'Midterm_Score', 'Final_Score', 'Assignments_Avg', 'Quizzes_Avg', 'Projects_Score', 'Total_Score']
numerical_scalar_vars = list(set(filtered_df.columns) - set(category_vars) - set(numerical_score_vars))
numerical_scalar_vars

['Sleep_Hours_per_Night', 'Stress_Level (1-10)', 'Study_Hours_per_Week', 'Age']

In [ ]:
class Feature_Embed(nn.Module):
    def __init__(self, cols_name, category_vars, feature_emb_size, num_variables, position_emb_dim, category_var_len):
        super().__init__()
        self.cols_name = cols_name
        self.category_vars = category_vars
        self.feature_emb_size = feature_emb_size
        self.category_var_len = category_var_len
        self.hidden_size = feature_emb_size + position_emb_dim

        # Category feature embedding
        self.cat_encoders = {
            'Gender': nn.Embedding(category_var_len['Gender'] + 1, feature_emb_size),
            'Department': nn.Embedding(category_var_len['Department'] + 1, feature_emb_size),
            'Grade': nn.Embedding(category_var_len['Grade'] + 1, feature_emb_size),
            'Extracurricular_Activities': nn.Embedding(category_var_len['Extracurricular_Activities'] + 1, feature_emb_size),
            'Internet_Access_at_Home': nn.Embedding(category_var_len['Internet_Access_at_Home'] + 1, feature_emb_size),
            'Parent_Education_Level': nn.Embedding(category_var_len['Parent_Education_Level'] + 1, feature_emb_size),
            'Family_Income_Level': nn.Embedding(category_var_len['Family_Income_Level'] + 1, feature_emb_size),
        }
        
        self.cat_decoders = {
            'Gender': nn.Linear(self.hidden_size, category_var_len['Gender'] + 1, bias=False),
            'Department': nn.Linear(self.hidden_size, category_var_len['Department'] + 1, bias=False),
            'Grade': nn.Linear(self.hidden_size, category_var_len['Grade'] + 1, bias=False),
            'Extracurricular_Activities': nn.Linear(self.hidden_size, category_var_len['Extracurricular_Activities'] + 1, bias=False),
            'Internet_Access_at_Home': nn.Linear(self.hidden_size, category_var_len['Internet_Access_at_Home'] + 1, bias=False),
            'Parent_Education_Level': nn.Linear(self.hidden_size, category_var_len['Parent_Education_Level'] + 1, bias=False),
            'Family_Income_Level': nn.Linear(self.hidden_size, category_var_len['Family_Income_Level'] + 1, bias=False),
        }
        
        # self.cat_encoders['Gender'].weight = self.cat_decoders['Gender'].weight        
        # self.cat_encoders['Department'].weight = self.cat_decoders['Department'].weight        
        # self.cat_encoders['Grade'].weight = self.cat_decoders['Grade'].weight        
        # self.cat_encoders['Extracurricular_Activities'].weight = self.cat_decoders['Extracurricular_Activities'].weight        
        # self.cat_encoders['Internet_Access_at_Home'].weight = self.cat_decoders['Internet_Access_at_Home'].weight        
        # self.cat_encoders['Parent_Education_Level'].weight = self.cat_decoders['Parent_Education_Level'].weight        
        # self.cat_encoders['Family_Income_Level'].weight = self.cat_decoders['Family_Income_Level'].weight        

        # numerical decoder
        self.numerical_decoder1 = nn.Linear(self.hidden_size, self.hidden_size * 2, bias = False)
        self.numerical_decoder2 = nn.Linear(self.hidden_size * 2, self.hidden_size * 4, bias = False)
        self.numerical_decoder3 = nn.Linear(self.hidden_size * 4, self.hidden_size * 2, bias = False)
        self.numerical_decoder4 = nn.Linear(self.hidden_size * 2, self.hidden_size, bias = False)
        self.numerical_decoder5 = nn.Linear(self.hidden_size, 1, bias = False)

        self.relu = nn.ReLU()

        self.cols_id_name = {}
        count = 0
        for col in self.cols_name:
            self.cols_id_name[count] = col
            count += 1
            
        # positional embedding 
        self.position_emb_dim = position_emb_dim
        self.pos_emb = nn.Embedding(num_variables, position_emb_dim)            

    def encode(self, unmasked_data, unmasked_idx, masked_idx):
        
        batch_size, num_unmask_cols = unmasked_data.shape
        _, num_mask_cols = masked_idx.shape
        device = unmasked_data.device
        
        # position info
        unmasked_pos_info = self.pos_emb(unmasked_idx.long()).float()
        masked_pos_info = self.pos_emb(masked_idx.long()).float()

        ''' Feature Encoding for Unmask '''
        
        # iterate every col in unmask
        # for category, do nn.Embedding
        # for numerical, repeat twice
        unmasked_emb = []
        for c in range(num_unmask_cols):
            unmask_attribute_value = unmasked_data[:, c]
            
            # make sure unmask_attribute_id is unique along one col dimension
            # use the unique id to see the choosen attribute is categorial or numerical
            unmask_attribute_id = unmasked_idx[:, c]
            assert len(torch.unique(unmask_attribute_id)) == 1, "unmask_attribute_id in one small batch has to be the same"
            unmask_attribute_id = torch.unique(unmask_attribute_id).item()

            # this col is categorial 
            if self.cols_id_name[unmask_attribute_id] in self.category_vars:
                # print(f"col_name: {self.cols_id_name[unmask_attribute_id]}")
                encoder = self.cat_encoders[self.cols_id_name[unmask_attribute_id]].to(device)
                unmask_attribute_emb = encoder(unmask_attribute_value.long()).float()
                unmasked_emb.append(unmask_attribute_emb)
            # this col is numerical
            else:
                unmask_attribute_value = torch.unsqueeze(unmask_attribute_value, dim = 1)
                unmask_attribute_emb = unmask_attribute_value.repeat(1, self.feature_emb_size)
                unmasked_emb.append(unmask_attribute_emb)

        unmasked_emb = torch.stack(unmasked_emb, dim = 1)
        
        ''' Positional Encoding for Unmask '''
        unmasked_emb = torch.cat([unmasked_emb, unmasked_pos_info], dim = 2) # (batch_size, num_unmask_vars, feature_emb_size + pos_emb_size)

        
        ''' Feature Encoding for Mask '''
        # create noise for categorial and numerical seperately
        masked_emb = []
        for c in range(num_mask_cols):
            mask_attribute_id = masked_idx[:, c]
            assert len(torch.unique(mask_attribute_id)) == 1, "mask_attribute_id in one small batch has to be the same"
            mask_attribute_id = torch.unique(mask_attribute_id).item()

            # this col is categorial 
            if self.cols_id_name[mask_attribute_id] in self.category_vars:
                # get col name
                col_name = self.cols_id_name[mask_attribute_id]
                encoder = self.cat_encoders[col_name].to(device)
                
                # create categorial mask id
                category_mask_id = self.category_var_len[col_name]
                categorial_latent = torch.ones(batch_size).to(device) * category_mask_id

                categorial_emb = encoder(categorial_latent.long()).float()
                masked_emb.append(categorial_emb)
                
            # this col is numerical
            else:
                # create rand(0-1) as numerical latent
                numerical_latent = torch.rand(batch_size).to(device)
                numerical_latent = torch.unsqueeze(numerical_latent, dim = 1)
                numerical_emb = numerical_latent.repeat(1, self.feature_emb_size)
                masked_emb.append(numerical_emb)

        masked_emb = torch.stack(masked_emb, dim = 1)
        
        ''' Positional Encoding for Mask '''
        masked_emb = torch.cat([masked_emb, masked_pos_info], dim = 2) # (batch_size, num_mask_vars, feature_emb_size + pos_emb_size)

        return unmasked_emb, masked_emb
    
    def decode(self, all_emb, unmask_mask_id):
        
        
        '''
        unmask categorial
        unmask numercial
        mask categorial
        mask numercial
        '''
            
        # cols_id_name: {0: 'Gender', 1: 'Department', 2: 'Grade', 3: 'Extracurricular_Activities', 
        # 4: 'Internet_Access_at_Home', 5: 'Parent_Education_Level', 6: 'Family_Income_Level', 
        # 7: 'Attendance (%)', 8: 'Midterm_Score', 9: 'Final_Score', 10: 'Assignments_Avg', 
        # 11: 'Quizzes_Avg', 12: 'Projects_Score', 13: 'Total_Score', 14: 'Stress_Level (1-10)', 
        # 15: 'Sleep_Hours_per_Night', 16: 'Study_Hours_per_Week', 17: 'Age'}

        batch_size, num_cols, hidden_size = all_emb.shape
        device = all_emb.device
        
        ''' decode for unmask and mask '''
        categorial_preds = []
        numerical_preds = []
        
        categorial_col_name = []
        numerical_col_name = []
        
        for c in range(num_cols):
            # attribute value
            attribute_value = all_emb[:, c, :]

            # attribute id
            attribute_id = unmask_mask_id[:, c]
            assert len(torch.unique(attribute_id)) == 1, "attribute_id in one small batch has to be the same"
            attribute_id = torch.unique(attribute_id).item()

            # this col is categorial 
            if self.cols_id_name[attribute_id] in self.category_vars:
                col_name = self.cols_id_name[attribute_id]
                decoder = self.cat_decoders[col_name].to(device)
                cat_pred = decoder(attribute_value)
                categorial_preds.append(cat_pred)
                categorial_col_name.append(col_name)
                
            # this col is numerical
            else:
                ori_attribute_value = attribute_value
                
                attribute_value = self.numerical_decoder1(attribute_value).to(device)
                attribute_value = self.relu(attribute_value)
                
                attribute_value = self.numerical_decoder2(attribute_value).to(device)
                attribute_value = self.relu(attribute_value)
                
                attribute_value = self.numerical_decoder3(attribute_value).to(device)
                attribute_value = self.relu(attribute_value)
                
                attribute_value = self.numerical_decoder4(attribute_value).to(device)
                attribute_value = self.relu(attribute_value)
                
                # residual connection
                attribute_value = attribute_value + ori_attribute_value
                
                num_pred = self.numerical_decoder5(attribute_value).to(device)
                numerical_preds.append(num_pred)
                
                col_name = self.cols_id_name[attribute_id]
                numerical_col_name.append(col_name)

        numerical_preds = torch.stack(numerical_preds, dim = 1)
        numerical_preds = torch.squeeze(numerical_preds)
    
        return categorial_preds, numerical_preds, categorial_col_name, numerical_col_name
    
    
    def reorder_label_fn(self, label, unmask_mask_id):
        
        batch_size, num_cols = label.shape
        device = label.device
        
        ''' reorder for unmask and mask '''
        categorial_labels = []
        numerical_labels = []
        
        for c in range(num_cols):
            # attribute value
            attribute_value = label[:, c]

            # attribute id
            attribute_id = unmask_mask_id[:, c]
            assert len(torch.unique(attribute_id)) == 1, "attribute_id in one small batch has to be the same"
            attribute_id = torch.unique(attribute_id).item()

            # this col is categorial 
            if self.cols_id_name[attribute_id] in self.category_vars:
                categorial_labels.append(attribute_value)
                
            # this col is numerical
            else:
                numerical_labels.append(attribute_value)
              
        numerical_labels = torch.stack(numerical_labels, dim = 1)

        return categorial_labels, numerical_labels
    

In [18]:
class tableMET(nn.Module):
    def __init__(self, position_emb_dim, layers_encode, layers_decode, num_head, num_variables, cols_name, category_vars, feature_emb_size, category_var_len, dropout):
        super().__init__()
        self.num_variables = num_variables
        self.unmask_ratio = ((100 - MASK_RATIO) * num_variables) // 100
        self.mask_ratio = num_variables - ((100 - MASK_RATIO) * num_variables) // 100

        # feature embedding
        self.feature_emb_fn = Feature_Embed(cols_name, category_vars, feature_emb_size, num_variables, position_emb_dim, category_var_len)

        # encoder and decoder
        self.transformer_encode = Transformer(position_emb_dim + feature_emb_size, layers_encode, num_head, dropout)
        self.transformer_decode = Transformer(position_emb_dim + feature_emb_size, layers_decode, num_head, dropout)
        
    def forward(self, unmasked_data, unmasked_idx, masked_idx, label):
        
        ''' feature & position encoding for Unmask and Mask '''
        # unmask_emb: [batch_size, num_unmask = 12, hidden_size(position_emb_dim + feature_emb_size)]: ([4, 12, 64])
        # mask_emb: [batch_size, num_mask = 6, hidden_size(position_emb_dim + feature_emb_size)]: ([4, 6, 64])
        unmask_emb, mask_emb = self.feature_emb_fn.encode(unmasked_data, unmasked_idx, masked_idx)
        
        # transformer (encoder) only for unmasked part
        # unmask_emb: [batch_size, num_unmask = 12, hidden_size(position_emb_dim + feature_emb_size)]: ([4, 12, 64])
        unmask_emb = self.transformer_encode(unmask_emb)
        
        # combine unmask_emb and mask_emb
        # all_emb: [batch_size, num_vars = 18, hidden_size(position_emb_dim + feature_emb_size)]: ([4, 18, 64])
        all_emb = torch.cat([unmask_emb, mask_emb], dim = 1)
        
        # transformer (decoder) for both unmasked and masked
        # all_emb: [batch_size, num_vars = 18, hidden_size(position_emb_dim + feature_emb_size)]: ([4, 18, 64])
        all_emb = self.transformer_decode(all_emb)

        ''' feature & position decoding for Unmask and Mask '''
        unmask_mask_id = torch.cat([unmasked_idx, masked_idx], dim = 1)
        categorial_preds, numerical_preds, categorial_col_name, numerical_col_name = self.feature_emb_fn.decode(all_emb, unmask_mask_id)

        ''' reorder label: make label' variable order align as pred '''
        categorial_labels, numerical_labels = self.feature_emb_fn.reorder_label_fn(label, unmask_mask_id)
            

        return categorial_preds, numerical_preds, categorial_labels, numerical_labels, categorial_col_name, numerical_col_name
    
        

In [19]:
def linear_warmup_decay_lr(lr_init, lr_final, num_warmup_steps, num_training_steps):
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return current_step / num_warmup_steps 
        else:
            progress = (current_step - num_warmup_steps) / (num_training_steps - num_warmup_steps)
            return (1 - progress) * (1 - lr_final / lr_init) + (lr_final / lr_init)  
    return lr_lambda

In [20]:
LEARNING_RATE = 1e-4
EPOCHS = 1200 
position_emb_dim = 512
layers_encode = 6
layers_decode = 2
num_head = 8
dropout = 0.3
num_variables = 18
unmask_ratio = ((100 - MASK_RATIO) * num_variables) // 100
cols_name = processed_df_train.columns
feature_emb_size = 512
category_var_len = category_var_len

is_power_of_two = lambda n: n > 0 and (n & (n - 1)) == 0
assert is_power_of_two(position_emb_dim + feature_emb_size), "position_emb_dim + feature_emb_size should be power term of 2"

model = tableMET(position_emb_dim, layers_encode, layers_decode, num_head, num_variables, cols_name, category_vars, feature_emb_size, category_var_len, dropout).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, betas=(0.9, 0.95), eps=1e-6, weight_decay=1e-3)

scheduler = LambdaLR(optimizer, lr_lambda=linear_warmup_decay_lr(lr_init = LEARNING_RATE, lr_final = LEARNING_RATE * 1e-2, num_warmup_steps = 10, num_training_steps = EPOCHS))


In [21]:
MSE_loss_fn = nn.MSELoss()
CE_loss_fn = nn.CrossEntropyLoss()

def loss_fn(categorial_pred, numerical_pred, categorial_label, numerical_label):
    
    num_numerical = 12
    num_category = 7
    ratio_numerical = num_numerical / (num_numerical + num_category)
    ratio_category = 1 / (num_numerical + num_category)
    
    total_loss = torch.zeros(1).to(device)
    
    mse_loss = MSE_loss_fn(numerical_pred, numerical_label)
    mse_loss = mse_loss * 1
    total_loss += (mse_loss * ratio_numerical)
    
    for i in range(num_category):
        pred = categorial_pred[i]
        label = categorial_label[i]        
        loss = CE_loss_fn(pred, label.long())
        total_loss += (loss * ratio_category)
        
    return total_loss, mse_loss, (total_loss - mse_loss).item()

In [22]:
train_LOSS = []
valid_LOSS = []

for epoch in tqdm(range(EPOCHS), desc="iterate epoch"):
    
    losses = []
    mse_losses = []
    ce_losses = []
    
    val_losses = []
    val_mse_losses = []
    val_ce_losses = []

    model.train()
    for data, label in train_dataloader:
        unmasked_data = data[0].float().to(device)
        unmasked_idx = data[1].long().to(device)
        masked_idx = data[2].long().to(device)
        latent = data[3].float().to(device)
        label = label.float().to(device)
        
        categorial_pred, numerical_pred, categorial_label, numerical_label, categorial_col_name, numerical_col_name = model(unmasked_data, unmasked_idx, masked_idx, label)
        
        loss, mse_loss, ce_loss = loss_fn(categorial_pred, numerical_pred, categorial_label, numerical_label)
        
        losses.append(loss.item())
        mse_losses.append(mse_loss.item())
        ce_losses.append(ce_loss)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    scheduler.step()
    
    losses = np.mean(losses)
    mse_losses = np.mean(mse_losses)
    ce_losses = np.mean(ce_losses)
    train_LOSS.append(losses)
    
    if epoch % 50 == 0:
        print(f'total variables are {num_variables}: first {unmask_ratio} unmasked, latter {num_variables - unmask_ratio}')
        print(f"epoch: {epoch}, loss: {losses}, mse: {mse_losses}, ce: {ce_losses}")
    
    with torch.no_grad():
        model.eval()
        for data, label in valid_dataloader:
            unmasked_data = data[0].float().to(device)
            unmasked_idx = data[1].long().to(device)
            masked_idx = data[2].long().to(device)
            latent = data[3].float().to(device)
            label = label.float().to(device)

            categorial_pred, numerical_pred, categorial_label, numerical_label, categorial_col_name, numerical_col_name = model(unmasked_data, unmasked_idx, masked_idx, label)
            
            loss, mse_loss, ce_loss = loss_fn(categorial_pred, numerical_pred, categorial_label, numerical_label)

            val_losses.append(loss.item())
            val_mse_losses.append(mse_loss.item())
            val_ce_losses.append(ce_loss)            
            
    val_losses = np.mean(val_losses)
    val_mse_losses = np.mean(val_mse_losses)
    val_ce_losses = np.mean(val_ce_losses)
    valid_LOSS.append(val_losses)
    
    if epoch % 50 == 0:
        print(f"epoch: {epoch}, val_loss: {val_losses}, val_mse: {val_mse_losses}, val_ce_losses: {val_ce_losses}")
        print()        
        
    

iterate epoch:   0%|          | 0/1200 [00:01<?, ?it/s]

numerical_preds: torch.Size([256, 11])


RuntimeError: No active exception to reraise

In [ ]:
plt.plot(range(len(train_LOSS)), train_LOSS, color = 'blue')
plt.plot(range(len(valid_LOSS)), valid_LOSS, color = 'red')

plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Inference

In [ ]:
def reverse_norm(df):
    
    new_df = pd.DataFrame()
    

    # Category 
    for cat_name in category_vars:
        cat_var = df[cat_name]
        new_df[f'{cat_name}'] = cat_var
    
    # Numerical score
    for num_name in numerical_score_vars:
        num_var = df[num_name]
        
        # reverse (0 - 100) min max norm
        num_var = (num_var * 100) + 0
        
        new_df[num_name] = num_var.values
    
    # Numerical scalar
    for num_name in numerical_scalar_vars:
        num_var = df[num_name]
        
        # reverse (0 - 10) min max norm
        num_var = (num_var * 10) + 0
        # reverse
        num_var = np.exp(num_var)
        new_df[num_name] = num_var.values
        
    return new_df

In [ ]:
select_sample_id = 5


In [ ]:
unmasked_idx_one = np.array(unmasked_idx[select_sample_id].detach().cpu(), dtype=int)
masked_idx_one = np.array(masked_idx[select_sample_id].detach().cpu(), dtype=int)


categorial_pred_one = []
for pred in categorial_pred:
    argmax_pred = torch.argmax(pred, dim = 1)
    categorial_pred_one.append(argmax_pred)
categorial_pred_one = torch.stack(categorial_pred_one, dim = 1)
categorial_pred_one = np.array(categorial_pred_one[select_sample_id].detach().cpu())

categorial_label_one = torch.stack(categorial_label, dim = 1)
categorial_label_one = np.array(categorial_label_one[select_sample_id].detach().cpu())

numerical_pred_one = np.array(numerical_pred[select_sample_id].detach().cpu())
numerical_label_one = np.array(numerical_label[select_sample_id].detach().cpu())


print(f"categorial_pred_one: {categorial_pred_one.shape}")
print(f"categorial_label_one: {categorial_label_one.shape}")
print(f"numerical_pred_one: {numerical_pred_one.shape}")
print(f"numerical_label_one: {numerical_label_one.shape}")

In [ ]:
pred = np.concat([categorial_pred_one, numerical_pred_one], axis = 0)
label = np.concat([categorial_label_one, numerical_label_one], axis = 0)

res = pd.DataFrame([pred, label], columns = categorial_col_name + numerical_col_name)
reverse_res = reverse_norm(res)

In [ ]:
reverse_res[cols_name[masked_idx_one]]

In [ ]:
reverse_res[cols_name[unmasked_idx_one]]

In [ ]:
category_vars = ['Gender', 'Department', 'Grade', 'Extracurricular_Activities', 'Internet_Access_at_Home', 'Parent_Education_Level', 'Family_Income_Level']
numerical_score_vars = ['Attendance (%)', 'Midterm_Score', 'Final_Score', 'Assignments_Avg', 'Quizzes_Avg', 'Projects_Score', 'Total_Score']
numerical_scalar_vars = list(set(filtered_df.columns) - set(category_vars) - set(numerical_score_vars))
numerical_scalar_vars
# ['Study_Hours_per_Week', 'Age', 'Stress_Level (1-10)', 'Sleep_Hours_per_Night']
